In [16]:
# https://github.com/buildingSMART/Sample-Test-Files/tree/master/IFC%202x3/Schependomlaan
# https://github.com/buildingSMART/Sample-Test-Files/blob/master/IFC%202x3/Medical-Dental%20Clinic/Clinic_Structural.ifc

In [29]:
import ifcopenshell
import ifcopenshell.geom


ifc_file = ifcopenshell.open('Clinic_Structural.ifc')

element = ifc_file.by_type('IfcColumn')[100]

settings = ifcopenshell.geom.settings()
shape = ifcopenshell.geom.create_shape(settings, element)

# The GUID of the element we processed
print(shape.guid)

# The ID of the element we processed
print(shape.id)

# The element we are processing
print(ifc_file.by_guid(shape.guid))

# A unique geometry ID, useful to check whether or not two geometries are
# identical for caching and reuse. The naming scheme is:
# IfcShapeRepresentation.id{-layerset-LayerSet.id}{-material-Material.id}{-openings-[Opening n.id ...]}{-world-coords}
print("verts:")
print(shape.geometry.verts )


# A 4x4 matrix representing the location and rotation of the element, in the form:
# [ [ x_x, y_x, z_x, x   ]
#   [ x_y, y_y, z_y, y   ]
#   [ x_z, y_z, z_z, z   ]
#   [ 0.0, 0.0, 0.0, 1.0 ] ]
# The position is given by the last column: (x, y, z)
# The rotation is described by the first three columns, by explicitly specifying the local X, Y, Z axes.
# The first column is a normalised vector of the local X axis: (x_x, x_y, x_z)
# The second column is a normalised vector of the local Y axis: (y_x, y_y, y_z)
# The third column is a normalised vector of the local Z axis: (z_x, z_y, z_z)
# The axes follow a right-handed coordinate system.
# Objects are never scaled, so the scale factor of the matrix is always 1.
matrix = shape.transformation.matrix.data

# Indices of vertices per triangle face e.g. [f1v1, f1v2, f1v3, f2v1, f2v2, f2v3, ...]
faces = shape.geometry.faces
print("faces:")
print(faces)

# Indices of vertices per edge e.g. [e1v1, e1v2, e2v1, e2v2, ...]
edges = shape.geometry.edges

# X Y Z of vertices in flattened list e.g. [v1x, v1y, v1z, v2x, v2y, v2z, ...]
verts = shape.geometry.verts

# Since the lists are flattened, you may prefer to group them like so depending on your geometry kernel
grouped_verts = [[verts[i], verts[i + 1], verts[i + 2]] for i in range(0, len(verts), 3)]
grouped_edges = [[edges[i], edges[i + 1]] for i in range(0, len(edges), 2)]
grouped_faces = [[faces[i], faces[i + 1], faces[i + 2]] for i in range(0, len(faces), 3)]


print(grouped_verts)

# A list of styles that are relevant to this shape
styles = shape.geometry.materials

for style in styles:
    # Each style is named after the entity class if a default
    # material is applied. Otherwise, it is named "surface-style-{SurfaceStyle.name}"
    # All non-alphanumeric characters are replaced with a "-".
    print(style.original_name())

    # A more human readable name
    print(style.name)

    # Each style may have diffuse colour RGB codes
    if style.has_diffuse:
        print(style.diffuse)

    # Each style may have transparency data
    if style.has_transparency:
        print(style.transparency)

# Indices of material applied per triangle face e.g. [f1m, f2m, ...]
material_ids = shape.geometry.material_ids

3saVcraO9CJRqkF0xoeCHq
1069
#1069=IfcColumn('3saVcraO9CJRqkF0xoeCHq',#1,'M_W-Wide Flange-Column:W250X67:W250X67:160006',$,'W250X67',#220509,#218386,'160006')
verts:
(0.102, 0.1285, -0.305000000000353, 0.102, 0.1285, 4.570000000004097, -0.102, 0.1285, 4.570000000004097, -0.102, 0.1285, -0.305000000000353, -0.102, 0.1128, 4.570000000004097, -0.102, 0.1128, -0.305000000000353, -0.02175000000000004, 0.11279999999999998, 4.570000000004097, -0.02175000000000004, 0.11279999999999998, -0.305000000000353, -0.004449999999999992, 0.09550000000000003, -0.305000000000353, -0.004449999999999992, 0.09550000000000003, 4.570000000004097, -0.004883747119254472, 0.09934961215744431, 4.570000000004097, -0.006163238585288193, 0.10300618868673382, 4.570000000004097, -0.00822431535330315, 0.10628637357215615, 4.570000000004097, -0.010963626427843993, 0.10902568464669694, 4.570000000004097, -0.014243811313266339, 0.11108676141471185, 4.570000000004097, -0.017900387842555866, 0.11236625288074553, 4.57000000000

In [18]:
import ifcopenshell
import json
import pandas as pd

# Load IFC file
model = ifcopenshell.open('IFC Schependomlaan.ifc')

df = []

columns = model.by_type("IfcColumn")
num_columns = len(columns)

for x in range(num_columns):
    columns = model.by_type("IfcColumn")[x]
    # columns_type = ifcopenshell.util.element.get_type(columns)
     # Get all properties and quantities of the columns, including inherited type properties
    psets = ifcopenshell.util.element.get_psets(columns)
    
    y = psets['ArchiCADProperties']['Home Story']
    if y == '00 begane grond':
        # df.append(psets)
        if psets['AC_Equantity_Buis_ro_warmgew_kolom_18']['Hoogte kolom'] or psets['AC_Equantity_Buis_vk_warmgew_kolom_18']['Hoogte kolom'] != 'NaN':
            # df.append(psets['AC_Equantity_Buis_vk_warmgew_kolom_18']['Hoogte kolom'])
            df.append(psets["AC_Equantity_Buis_ro_warmgew_kolom_18"]['Hoogte kolom'])

df = pd.DataFrame.from_dict(df)
df

# g = psets['AC_Equantity_Buis_vk_warmgew_kolom_18']['Hoogte kolom']
# ['AC_Equantity_Buis_ro_warmgew_kolom_18']['Hoogte kolom']
# print(g)
# != NaN

KeyError: 'AC_Equantity_Buis_ro_warmgew_kolom_18'

In [ ]:
import ifcopenshell
import json
import pandas as pd

# Load IFC file
model = ifcopenshell.open('Clinic_Structural.ifc')

# Initialize empty lists
df = []
everything = []
list_base = []
list_top = []
material = []

columns = model.by_type("IfcColumn")
num_columns = len(columns)

for x in range(num_columns):
    columns = model.by_type("IfcColumn")[x]
    psets = ifcopenshell.util.element.get_psets(columns)

    y = psets['PSet_Revit_Constraints']['Base Level']
    if y == 'First Floor':
        df.append(psets)
        list_base.append(psets['PSet_Revit_Constraints']['Base Offset'])
        list_top.append(psets['PSet_Revit_Constraints']['Top Offset'])
        material.append(psets['PSet_Revit_Materials and Finishes']['Column Material'])

# test = pd.DataFrame({'base': list_base, 'top': list_top})
# df = pd.DataFrame.from_dict(df)
# df

###########################################################################################
#   PredefinedType attribute is specific to columns in IFC4 and newer versions. In IFC2X3, you can get the "Elevation Bottom" and "Elevation Top" properties of a column using the ObjectPlacement and Representation attributes. 


# # Get all columns in the IFC file
# columns = model.by_type("IfcColumn")

# # Iterate over each column and extract its properties
# for column in columns:
#     # Check if the object is an IfcColumn
#     if column.is_a("IfcColumn"):
#         # Get the Object Placement and Representation of the column
#         obj_placement = column.ObjectPlacement
#         representation = column.Representation

#         # Get the Elevation Bottom and Elevation Top properties from the Object Placement and Representation
#         if obj_placement.is_a("IfcLocalPlacement"):
#             relative_placement = obj_placement.RelativePlacement
#             if relative_placement.is_a("IfcAxis2Placement3D"):
#                 location = relative_placement.Location
#                 elevation_bottom = location.Coordinates[2]
#         # elevation_top = None  # Initialize elevation_top to None
#         # if representation.is_a("IfcShapeRepresentation"):
#         #     items = representation.Items
#         #     for item in items:
#         #         if item.is_a("IfcExtrudedAreaSolid"):
#         #             if item.Depth is not None:
#         #                 depth_position = item.Depth.Position
#         #                 elevation_top = depth_position.Coordinates[2]

#         print("Column ID:", column.id(), "Elevation Bottom:", elevation_bottom)
#         #print("Column ID:", column.id(), "Elevation Bottom:", elevation_bottom, "Elevation Top:", elevation_top)

######################################


# Get all columns in the IFC file
columns = model.by_type("IfcColumn")

# Iterate over each column and extract its properties
for column in columns:
    # Check if the object is an IfcColumn
    if column.is_a("IfcColumn"):
        # Get the Object Placement and Representation of the column
        obj_placement = column.ObjectPlacement
        representation = column.Representation

        # Get the Elevation Bottom and Elevation Top properties from the Object Placement and Representation
        if obj_placement.is_a("IfcLocalPlacement"):
            relative_placement = obj_placement.RelativePlacement
            if relative_placement.is_a("IfcAxis2Placement3D"):
                location = relative_placement.Location
                elevation_bottom = location.Coordinates[2]

        if representation.is_a("IfcProductDefinitionShape"):
            for shape in representation.Representations:
                if shape.is_a("IfcShapeRepresentation"):
                    for item in shape.Items:
                        if item.is_a("IfcExtrudedAreaSolid"):
                            height = item.Depth
            print(f"Height: {height}")
        else:
            print(f"Unsupported representation type: {representation.is_a()}")


        # Compute the elevation top
        # elevation_top = elevation_bottom + height
        # print(f"Elevation Bottom: {elevation_bottom}, Elevation Top: {elevation_top}")



######################################################################################################

# walls = model.by_type("IfcColumn")
# wall = walls[12]
# pset_for_wall = ifcopenshell.util.element.get_psets(wall)
# print(pset_for_wall)

# # Convert psets to JSON
# pset_json = json.dumps(pset_for_wall)

# # Save JSON to a file
# with open("pset_for_wall.json", "w") as f:
#     f.write(pset_json)

NameError: name 'height' is not defined

In [ ]:
import ifcopenshell
import json
import pandas as pd

# Load IFC file
model = ifcopenshell.open('1807_EP_AR_v18.ifc')

# Initialize empty lists
df = []
everything = []
list_base = []
list_top = []
material = []

columns = model.by_type("IfcColumn")
num_columns = len(columns)

for x in range(num_columns):
    columns = model.by_type("IfcColumn")[x]
    psets = ifcopenshell.util.element.get_psets(columns)

    # y = psets['PSet_Revit_Constraints']['Base Level']
    # if y == 'First Floor':
    df.append(psets)
        # list_base.append(psets['PSet_Revit_Constraints']['Base Offset'])
        # list_top.append(psets['PSet_Revit_Constraints']['Top Offset'])
        # material.append(psets['PSet_Revit_Materials and Finishes']['Column Material'])

# test = pd.DataFrame({'base': list_base, 'top': list_top})
df = pd.DataFrame.from_dict(df)
df

# walls = model.by_type("IfcColumn")
# wall = walls[12]
# pset_for_wall = ifcopenshell.util.element.get_psets(wall)
# pset_for_wall

,Ehituskonstruktsioonid,AC_Pset_RenovationAndPhasing,Pset_ColumnCommon,Component Properties,Component Quantities
0,"{'01 Nimetus': 'Varikatuse post', '02 Materjal...","{'Renovation Status': 'New', 'id': 39693}","{'LoadBearing': True, 'IsExternal': True, 'Ref...","{'Component 1': {'id': 39709, 'type': 'IfcComp...","{'Component 1': {'id': 39724, 'type': 'IfcPhys..."
1,"{'01 Nimetus': 'Varikatuse post', '02 Materjal...","{'Renovation Status': 'New', 'id': 39787}","{'LoadBearing': True, 'IsExternal': True, 'Ref...","{'Component 1': {'id': 39803, 'type': 'IfcComp...","{'Component 1': {'id': 39818, 'type': 'IfcPhys..."
2,"{'01 Nimetus': 'Varikatuse post', '02 Materjal...","{'Renovation Status': 'New', 'id': 40352}","{'LoadBearing': True, 'IsExternal': True, 'Ref...","{'Component 1': {'id': 40368, 'type': 'IfcComp...","{'Component 1': {'id': 40383, 'type': 'IfcPhys..."
3,"{'01 Nimetus': 'Varikatuse post', '02 Materjal...","{'Renovation Status': 'New', 'id': 47404}","{'LoadBearing': True, 'IsExternal': True, 'Ref...","{'Component 1': {'id': 47420, 'type': 'IfcComp...","{'Component 1': {'id': 47435, 'type': 'IfcPhys..."
4,"{'01 Nimetus': 'Varikatuse post', '02 Materjal...","{'Renovation Status': 'New', 'id': 47586}","{'LoadBearing': True, 'IsExternal': True, 'Ref...","{'Component 1': {'id': 47602, 'type': 'IfcComp...","{'Component 1': {'id': 47617, 'type': 'IfcPhys..."
5,"{'01 Nimetus': 'Varikatuse post', '02 Materjal...","{'Renovation Status': 'New', 'id': 53961}","{'LoadBearing': True, 'IsExternal': True, 'Ref...","{'Component 1': {'id': 53977, 'type': 'IfcComp...","{'Component 1': {'id': 53992, 'type': 'IfcPhys..."
6,"{'01 Nimetus': 'Varikatuse post', '02 Materjal...","{'Renovation Status': 'New', 'id': 55803}","{'LoadBearing': True, 'IsExternal': True, 'Ref...","{'Component 1': {'id': 55819, 'type': 'IfcComp...","{'Component 1': {'id': 55834, 'type': 'IfcPhys..."
7,"{'01 Nimetus': 'Varikatuse post', '02 Materjal...","{'Renovation Status': 'New', 'id': 58633}","{'LoadBearing': True, 'IsExternal': True, 'Ref...","{'Component 1': {'id': 58649, 'type': 'IfcComp...","{'Component 1': {'id': 58664, 'type': 'IfcPhys..."
8,"{'01 Nimetus': 'Varikatuse post', '02 Materjal...","{'Renovation Status': 'New', 'id': 62555}","{'LoadBearing': True, 'IsExternal': True, 'Ref...","{'Component 1': {'id': 62571, 'type': 'IfcComp...","{'Component 1': {'id': 62586, 'type': 'IfcPhys..."
9,"{'01 Nimetus': 'Varikatuse post', '02 Materjal...","{'Renovation Status': 'New', 'id': 62645}","{'LoadBearing': True, 'IsExternal': True, 'Ref...","{'Component 1': {'id': 62661, 'type': 'IfcComp...","{'Component 1': {'id': 62676, 'type': 'IfcPhys..."


In [ ]:
# import ifcopenshell
# import json
# import pandas as pd

# # Load IFC file
# model = ifcopenshell.open('IFC Schependomlaan.ifc')

# df = []

# columns = model.by_type("IfcColumn")
# num_columnss = len(columns)

# for x in range(num_columnss):
#     columns = model.by_type("IfcColumn")[x]
#     # columns_type = ifcopenshell.util.element.get_type(columns)
#      # Get all properties and quantities of the columns, including inherited type properties
#     psets = ifcopenshell.util.element.get_psets(columns)
#     y = psets['ArchiCADProperties']['Home Story']
#     if y == '00 begane grond':
#         df.append(psets)
        
# df = pd.DataFrame.from_dict(df)
# df

# g = psets['ArchiCADProperties']['Home Story']['Hoogte kolom']
# print(g)
# print(psets.keys('ArchiCADProperties'))
# x = psets['ArchiCADProperties']['Home Story']
# if x == '00 begane grond':
#     print(x)
    
# with open('my_dict.json', 'w') as file:
#     json.dump(psets, file)

# LoadBearing=  column.get('LoadBearing', False)
                    # 'Diameter doorsnede (mm)': prop.get('Diameter doorsnede (mm)', 0),
                    # 'Elasticiteitsmodulus (cm3)': prop.get('Elasticiteitsmodulus (cm3)', 0),
                    # 'Height (Z Size)': prop.get('Height (Z Size)', 0)

TypeError: string indices must be integers